In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [2]:
import math
from sklearn.metrics import mean_squared_error

# Funções Auxiliares

In [3]:
def normalize(df):
    mindf = df.min()
    maxdf = df.max()
    return (df-mindf)/(maxdf-mindf)

In [4]:
def denormalize(norm, _min, _max):
    return [(n * (_max-_min)) + _min for n in norm]

In [5]:
def split_data(df, interval):
    sample_df = df.loc[interval]

    week = (sample_df.index.day - 1) // 7 + 1

    # PARA OS TESTES:
    # 2 SEMANAS PARA TREINAMENTO
    train_df = sample_df.loc[week <= 2]

    # 1 SEMANA PARA VALIDACAO
    validation_df = sample_df.loc[week == 3]

    # 1 SEMANA PARA TESTES
    test_df = sample_df.loc[week > 3]
    
    return (train_df, validation_df, test_df)

In [6]:
def calculate_rmse(test, forecast, order, step):
    rmse = math.sqrt(mean_squared_error(test.iloc[(order):], forecast[:-step]))
    print("RMSE : "+str(rmse))
    return rmse

In [7]:
def reconstruct_ssa_series(clean, residual):
    return [r + c for r, c in zip(residual,clean)]

In [8]:
def save_obj(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [9]:
def difference(raw_df, interval=1):
    df_diff = pd.DataFrame(columns=raw_df.columns, index=raw_df.index[1:])
    
    for col in raw_df.columns:
        raw_array = raw_df[col]
        diff = []
        for i in range(interval, len(raw_array)):
            value = raw_array[i] - raw_array[i - interval]
            diff.append(value)
        
        df_diff[col] = diff
    return df_diff

In [10]:
def inverse_difference(raw_series, diff_series):
    inverted = []
    for i in range(len(diff_series)):
        interval = len(raw_series)-i
        value = diff_series[i] + raw_series[-interval]
        inverted.append(value)
        
    return inverted

# Load Dataset
Split the data into train, validation and test subsets

In [9]:
#Set target and input variables 
target_station = 'DHHL_3'

#All neighbor stations with residual correlation greater than .90
neighbor_stations_90 = ['DHHL_3',  'DHHL_4','DHHL_5','DHHL_10','DHHL_11','DHHL_9','DHHL_2', 'DHHL_6','DHHL_7','DHHL_8']

In [10]:
df = pd.read_pickle("df_oahu.pkl")

In [11]:
## Remove columns with many corrupted or missing values
df.drop(columns=['AP_1', 'AP_7'], inplace=True)

In [12]:
#Normalize Data

# Save Min-Max for Denorm
min_raw = df[target_station].min()

max_raw = df[target_station].max()

# Perform Normalization
norm_df = normalize(df)

In [13]:
# Split data
interval = ((df.index >= '2010-06') & (df.index < '2011-06'))
#interval = ((df.index >= '2010-11') & (df.index <= '2010-12'))

(train_df, validation_df, test_df) = split_data(df, interval)
(norm_train_df, norm_validation_df, norm_test_df) = split_data(norm_df, interval)

## Forecasting with Raw Time Series

For each dataset, all the time series were normalized

## Persistence

In [16]:
def persistence_forecast(train, test, step):
    predictions = []
    
    for t in np.arange(0,len(test), step):
        yhat = [test.iloc[t]]  * step
        predictions.extend(yhat)
        
    return predictions

In [22]:
step = 1
persistence_order = 1

forecast = persistence_forecast(norm_train_df[target_station], norm_test_df[target_station],step)
forecast = denormalize(forecast, min_raw, max_raw)

rmse = calculate_rmse(test_df[target_station], forecast, persistence_order, step)

RMSE : 116.58118645284732


In [23]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_persistence_1")

## SARIMA

In [14]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

/Users/cseveriano/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [15]:
def sarima_forecast(train, test, arima_order, sarima_order, step):

    predictions = []
    window_size = sarima_order[3] * 5
    
    for date in train.index.to_period('M').unique():
        
        history = list(train[str(date)].iloc[-window_size:])
        
        model = SARIMAX(history, order=arima_order, seasonal_order=sarima_order,enforce_invertibility=False,enforce_stationarity=False)
        model_fit = model.fit(disp=True,enforce_invertibility=False,  method='powell', maxiter=200)
        
        #save the state parameter
        est_params = model_fit.params
        est_state = model_fit.predicted_state[:, -1]
        est_state_cov = model_fit.predicted_state_cov[:, :, -1]

        print("Predicting : "+str(date))
        
        st = 0
        test_date = test[str(date)]
        
        for t in np.arange(1,len(test_date)+1,step):
            obs = test_date.iloc[st:t].values
            history.extend(obs)
            history = history[-window_size:]
            
            mod_updated = SARIMAX(history, order=arima_order, seasonal_order=sarima_order,enforce_invertibility=False,enforce_stationarity=False)
            mod_updated.initialize_known(est_state, est_state_cov)
            mod_frcst = mod_updated.smooth(est_params)

        
            yhat = mod_frcst.forecast(step)   
            predictions.extend(yhat)
            
            est_params = mod_frcst.params
            est_state = mod_frcst.predicted_state[:, -1]
            est_state_cov = mod_frcst.predicted_state_cov[:, :, -1]
            
            st = t
                
    return predictions

In [16]:
order = 1
step = 1
arima_order = (2, 1, 2)
sarima_order = (1, 1, 1, 61)
forecast = sarima_forecast(norm_train_df[target_station], norm_test_df[target_station], arima_order, sarima_order, step)
forecast = denormalize(forecast, min_raw, max_raw)
rmse = calculate_rmse(test_df[target_station], forecast, order, step)

Optimization terminated successfully.
         Current function value: -0.619265
         Iterations: 8
         Function evaluations: 947
Predicting : 2010-06
Optimization terminated successfully.
         Current function value: 18.517468
         Iterations: 1
         Function evaluations: 88
Predicting : 2010-07
Optimization terminated successfully.
         Current function value: 18.934954
         Iterations: 1
         Function evaluations: 91
Predicting : 2010-08
Optimization terminated successfully.
         Current function value: 19.126512
         Iterations: 4
         Function evaluations: 415
Predicting : 2010-09
Optimization terminated successfully.
         Current function value: -0.728150
         Iterations: 23
         Function evaluations: 2698
Predicting : 2010-10
Optimization terminated successfully.
         Current function value: -0.790575
         Iterations: 8
         Function evaluations: 964
Predicting : 2010-11
Optimization terminated successfully.
  

In [194]:
result = {'rmse': rmse, 'final': forecast}

RMSE : 178.31337062654006


In [199]:
save_obj(result, name="oahu_raw_sarima_1")

## Vector Autoregressive - VAR

In [27]:
from statsmodels.tsa.api import VAR, DynamicVAR

In [28]:
def var_forecast(train, test, target, order, step):
    model = VAR(train.values)
    results = model.fit(maxlags=order)
    lag_order = results.k_ar
    print("Lag order:" + str(lag_order))
    forecast = []

    for i in np.arange(0,len(test)-lag_order+1,step) :
        forecast.extend(results.forecast(test.values[i:i+lag_order],step))

    forecast_df = pd.DataFrame(columns=test.columns, data=forecast)
    return forecast_df[target].values

In [30]:
var_order = 4
step = 1

forecast = var_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], target_station, var_order, step)
forecast = denormalize(forecast, min_raw, max_raw)

Lag order:4


In [31]:
rmse = calculate_rmse(test_df[target_station], forecast, var_order, step)

RMSE : 109.28142781608405


In [32]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_var_1")

## Long Short Term Memory - LSTM

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

Using TensorFlow backend.


## Multivariate LSTM

In [38]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [39]:
def lstm_multi_forecast(train_df, test_df, _order, _steps, _neurons, _epochs):

    
    nfeat = len(train_df.columns)
    nlags = _order
    nsteps = _steps
    nobs = nlags * nfeat
    
    train_reshaped_df = series_to_supervised(train_df, n_in=nlags, n_out=nsteps)
    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
    train_X = train_X.reshape((train_X.shape[0], nlags, nfeat))
    
    test_reshaped_df = series_to_supervised(test_df, n_in=nlags, n_out=nsteps)
    test_X, test_Y = test_reshaped_df.iloc[:,:nobs].values, test_reshaped_df.iloc[:,-nfeat].values
    test_X = test_X.reshape((test_X.shape[0], nlags, nfeat))
    
    # design network
    model = Sequential()
    model.add(LSTM(_neurons, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    
    # fit network
    model.fit(train_X, train_Y, epochs=_epochs, batch_size=72, verbose=False, shuffle=False)
    
    forecast = model.predict(test_X)
        
    return forecast

In [43]:
neurons = 50
lstm_order = 2
epochs = 100
steps = 1

forecast = lstm_multi_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], lstm_order, steps, neurons, epochs)
forecast = denormalize(forecast, min_raw, max_raw)

In [44]:
forecast.append(0) ## para manter o mesmo tamanho dos demais

In [45]:
rmse = calculate_rmse(test_df[target_station], forecast, lstm_order, step)

RMSE : 110.81399734978343


In [46]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_lstm_multi_1")

## Univariate LSTM

In [67]:
neurons = 50
lstm_order = 2
epochs = 100
steps = 1

forecast = lstm_multi_forecast(norm_train_df[[target_station]], norm_test_df[[target_station]], lstm_order, steps, neurons, epochs)
forecast = denormalize(forecast, min_raw, max_raw)

In [68]:
forecast.append(0) ## para manter o mesmo tamanho dos demais

In [69]:
rmse = calculate_rmse(test_df[target_station], forecast, lstm_order, step)
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_lstm_multi_1")

RMSE : 113.89297538695396


## Multi Layer Perceptron - MLP

In [47]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [48]:
def mlp_forecast(train_df, test_df, _order, _steps, _neurons, _epochs):

    
    nfeat = len(train_df.columns)
    nlags = _order
    nsteps = _steps
    nobs = nlags * nfeat
    
    train_reshaped_df = series_to_supervised(train_df, n_in=nlags, n_out=nsteps)
    train_X, train_Y = train_reshaped_df.iloc[:,:nobs].values, train_reshaped_df.iloc[:,-nfeat].values
    
    test_reshaped_df = series_to_supervised(test_df, n_in=nlags, n_out=nsteps)
    test_X, test_Y = test_reshaped_df.iloc[:,:nobs].values, test_reshaped_df.iloc[:,-nfeat].values
    
    # design network
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_dim=train_X.shape[1]))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # fit network
    history = model.fit(train_X, train_Y, epochs=_epochs, batch_size=72, verbose=False, shuffle=False)   

    forecast = model.predict(test_X)
        
    return forecast

In [49]:
neurons = 50
mlp_order = 2
epochs = 500
steps = 1

forecast = mlp_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], mlp_order, steps, neurons, epochs)
forecast = denormalize(forecast, min_raw, max_raw)

In [50]:
forecast.append(0) ## para manter o mesmo tamanho dos demais

In [51]:
rmse = calculate_rmse(test_df[target_station], forecast, mlp_order, steps)

RMSE : 109.39872643629796


In [52]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_mlp_multi_1")

## Univariate MLP

In [53]:
neurons = 50
mlp_order = 4
epochs = 500
steps = 1

forecast = mlp_forecast(norm_train_df[[target_station]], norm_test_df[[target_station]], mlp_order, steps, neurons, epochs)
forecast = denormalize(forecast, min_raw, max_raw)

forecast.append(0) ## para manter o mesmo tamanho dos demais

rmse = calculate_rmse(test_df[target_station], forecast, mlp_order, steps)

RMSE : 109.46746359249855


In [54]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_mlp_uni_1")

## High Order FTS

In [55]:
from pyFTS.partitioners import Grid, Entropy, Util as pUtil
from pyFTS.models import hofts
from pyFTS.common import Transformations

In [56]:
def hofts_forecast(train_df, test_df, _order, _partitioner, _npartitions):
    
    fuzzy_sets = _partitioner(data=train_df.values, npart=_npartitions)
    model_simple_hofts = hofts.HighOrderFTS()
    

    model_simple_hofts.fit(train_df.values, order=_order, partitioner=fuzzy_sets)

    
    forecast = model_simple_hofts.predict(test_df.values)

    return forecast

In [58]:
hofts_order = 2
#partitioner = Entropy.EntropyPartitioner
partitioner = Grid.GridPartitioner
nparts = 90


forecast = hofts_forecast(norm_train_df[target_station], norm_test_df[target_station], hofts_order, partitioner, nparts)
forecast = denormalize(forecast, min_raw, max_raw)

In [59]:
step = 1
rmse = calculate_rmse(test_df[target_station], forecast, hofts_order, step)

RMSE : 120.11789595500608


In [60]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_hofts_1")

## Clustered Multivariate

In [61]:
from models import KMeansPartitioner
from models import sthofts

In [62]:
import importlib
importlib.reload(sthofts)

<module 'models.sthofts' from '/Users/cseveriano/Google Drive/Doutorado/Codes/spatio-temporal-forecasting/src/models/sthofts.py'>

In [63]:
def sthofts_forecast(train_df, test_df, target, _order, npartitions):
    
    _partitioner = KMeansPartitioner.KMeansPartitioner(data=train_df.values, npart=npartitions, batch_size=1000, init_size=npartitions*3)
    model_sthofts = sthofts.SpatioTemporalHighOrderFTS()
    
    model_sthofts.fit(train_df.values, num_batches=100, order=_order, partitioner=_partitioner)
    forecast = model_sthofts.predict(test_df.values)
    forecast_df = pd.DataFrame(data=forecast, columns=test_df.columns)
    return forecast_df[target].values

In [64]:
sthofts_order = 2
nparts = 20


forecast = sthofts_forecast(norm_train_df[neighbor_stations_90], norm_test_df[neighbor_stations_90], target_station, sthofts_order, nparts)
forecast = denormalize(forecast, min_raw, max_raw)

In [65]:
step = 1
rmse = calculate_rmse(test_df[target_station], forecast, sthofts_order, step)

RMSE : 169.33198777653578


In [66]:
result = {'rmse': rmse, 'final': forecast}
save_obj(result, name="oahu_raw_sthofts_1")